In [19]:
import pandas as pd
import os

In [20]:
# Save df if does not exist
csv_file_path = '/Users/ajaykallepalli/data/SQL_df.csv'
if not os.path.exists(csv_file_path):
    splits = {'train': 'spider/train-00000-of-00001.parquet', 'validation': 'spider/validation-00000-of-00001.parquet'}
    df_train = pd.read_parquet("hf://datasets/xlangai/spider/" + splits["train"])
    df_train.to_csv(csv_file_path, index=False)
    print("Downloaded data")
else:
    print("Already downloaded")

/opt/homebrew/anaconda3/envs/llama_finetune/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Downloaded data


FileNotFoundError: hf://datasets/xlangai/spider/spider/test-00000-of-00001.parquet

In [21]:
#load csv if not loaded as a local variable

if 'df_train' not in locals():
    df_train = pd.read_csv(csv_file_path)

In [22]:
df_train.head()

,db_id,query,question,query_toks,query_toks_no_value,question_toks
0,department_management,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...,"['SELECT', 'count', '(', '*', ')', 'FROM', 'he...","['select', 'count', '(', '*', ')', 'from', 'he...","['How', 'many', 'heads', 'of', 'the', 'departm..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads...","['SELECT', 'name', ',', 'born_state', ',', 'ag...","['select', 'name', ',', 'born_state', ',', 'ag...","['List', 'the', 'name', ',', 'born', 'state', ..."
2,department_management,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac...","['SELECT', 'creation', ',', 'name', ',', 'budg...","['select', 'creation', ',', 'name', ',', 'budg...","['List', 'the', 'creation', 'year', ',', 'name..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...,"['SELECT', 'max', '(', 'budget_in_billions', '...","['select', 'max', '(', 'budget_in_billions', '...","['What', 'are', 'the', 'maximum', 'and', 'mini..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...,"['SELECT', 'avg', '(', 'num_employees', ')', '...","['select', 'avg', '(', 'num_employees', ')', '...","['What', 'is', 'the', 'average', 'number', 'of..."


In [23]:
# adding db_schema to the training data
# Getting unique db id list
unique_dbs = df_train['db_id'].unique()
unique_dbs

array(['department_management', 'farm', 'student_assessment', 'bike_1',
       'book_2', 'musical', 'twitter_1', 'product_catalog', 'flight_1',
       'allergy_1', 'store_1', 'journal_committee',
       'customers_card_transactions', 'race_track', 'coffee_shop',
       'chinook_1', 'insurance_fnol', 'medicine_enzyme_interaction',
       'university_basketball', 'phone_1', 'match_season', 'climbing',
       'body_builder', 'election_representative', 'apartment_rentals',
       'game_injury', 'soccer_1', 'performance_attendance', 'college_2',
       'debate', 'insurance_and_eClaims', 'customers_and_invoices',
       'wedding', 'theme_gallery', 'epinions_1', 'riding_club', 'gymnast',
       'small_bank_1', 'browser_web', 'wrestler', 'school_finance',
       'protein_institute', 'cinema', 'products_for_hire', 'phone_market',
       'gas_company', 'party_people', 'pilot_record',
       'cre_Doc_Control_Systems', 'company_1', 'local_govt_in_alabama',
       'formula_1', 'machine_repair', 'en

In [24]:
# creating a db with db ID and db schema as columns
df_schemas = pd.DataFrame(columns=['db_id', 'db_schema'])

In [25]:
schema_location = '/Users/ajaykallepalli/data/spider/database/'
schema_name = 'schema.sql'
for uniq_db in unique_dbs:
    schema_file = os.path.join(schema_location, uniq_db, schema_name)
    print(schema_file)
    if os.path.exists(schema_file):
        with open(schema_file, 'r') as f:
            schema_content = f.read()
        df_schemas = pd.concat([df_schemas, pd.DataFrame({'db_id': [uniq_db], 'db_schema': [schema_content]})], ignore_index=True)
    else:
        print(f"Schema file not found for {uniq_db}")

/Users/ajaykallepalli/data/spider/database/department_management/schema.sql
/Users/ajaykallepalli/data/spider/database/farm/schema.sql
/Users/ajaykallepalli/data/spider/database/student_assessment/schema.sql
/Users/ajaykallepalli/data/spider/database/bike_1/schema.sql
/Users/ajaykallepalli/data/spider/database/book_2/schema.sql
/Users/ajaykallepalli/data/spider/database/musical/schema.sql
/Users/ajaykallepalli/data/spider/database/twitter_1/schema.sql
Schema file not found for twitter_1
/Users/ajaykallepalli/data/spider/database/product_catalog/schema.sql
/Users/ajaykallepalli/data/spider/database/flight_1/schema.sql
/Users/ajaykallepalli/data/spider/database/allergy_1/schema.sql
/Users/ajaykallepalli/data/spider/database/store_1/schema.sql
/Users/ajaykallepalli/data/spider/database/journal_committee/schema.sql
/Users/ajaykallepalli/data/spider/database/customers_card_transactions/schema.sql
/Users/ajaykallepalli/data/spider/database/race_track/schema.sql
/Users/ajaykallepalli/data/spi

In [26]:
# Save the df_schemas DataFrame to a CSV file
csv_schemas_file_path = '/Users/ajaykallepalli/data/df_schemas.csv'

if not os.path.exists(csv_schemas_file_path):
    df_schemas.to_csv(csv_schemas_file_path, index=False)
    print(f"Schemas CSV file saved to {csv_schemas_file_path}")
else:
    print(f"Schemas CSV file already exists at {csv_schemas_file_path}")

# Print the shape of df_schemas
print("Shape of df_schemas:", df_schemas.shape)


Schemas CSV file saved to /Users/ajaykallepalli/data/df_schemas.csv
Shape of df_schemas: (127, 2)


In [27]:
df_schemas.head()

,db_id,db_schema
0,department_management,PRAGMA foreign_keys=ON;\nBEGIN TRANSACTION;\nC...
1,farm,"\nPRAGMA foreign_keys = ON;\n\nCREATE TABLE ""c..."
2,student_assessment,PRAGMA foreign_keys = ON;\n\nCREATE TABLE Addr...
3,bike_1,PRAGMA foreign_keys=OFF;\nBEGIN TRANSACTION;\n...
4,book_2,"PRAGMA foreign_keys = ON;\n\nCREATE TABLE ""pub..."


In [10]:
## Joining the df_schema to the train data.
df_train = df_train.merge(df_schemas, on='db_id', how='left')

# Check for any missing schemas
missing_schemas = df_train[df_train['db_schema'].isnull()]
if not missing_schemas.empty:
    print(f"Warning: {len(missing_schemas)} rows have missing schemas.")
    print("Unique db_ids with missing schemas:", missing_schemas['db_id'].unique())

# Verify the merge
print("Shape of df_train after merge:", df_train.shape)
print("Columns in df_train:", df_train.columns)

# Check for null values in the merged dataframe
null_counts = df_train.isnull().sum()
print("Null value counts:\n", null_counts)



Unique db_ids with missing schemas: ['twitter_1' 'chinook_1' 'college_2' 'epinions_1' 'small_bank_1'
 'company_1' 'formula_1' 'inn_1' 'icfp_1' 'college_1' 'student_1' 'wine_1'
 'flight_4']
Shape of df_train after merge: (7000, 7)
Columns in df_train: Index(['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value',
       'question_toks', 'db_schema'],
      dtype='object')
Null value counts:
 db_id                    0
query                    0
question                 0
query_toks               0
query_toks_no_value      0
question_toks            0
db_schema              984
dtype: int64


In [15]:
# Remove all NA values from df_train
df_schema_train = df_train.dropna()
# Group by 'db_id' and keep only the first 20 rows for each schema
df_schema_train = df_schema_train.groupby('db_id').apply(lambda x: x.head(20)).reset_index(drop=True)

# Print the shape of the new dataframe after keeping only 20 of each schema
print("Shape of df_schema_train after keeping 20 of each schema:", df_schema_train.shape)

# Print the number of unique schemas
print("Number of unique schemas:", df_schema_train['db_id'].nunique())

# Define the path for the CSV file
csv_schema_file_path = '/Users/ajaykallepalli/data/df_schema_train.csv'

# Save the new dataframe to the same folder
if not os.path.exists(csv_schema_file_path):
    df_schema_train.to_csv(csv_schema_file_path, index=False)
    print(f"File saved to {csv_schema_file_path}")
else:
    print(f"File already exists at {csv_schema_file_path}")

# Print the shape of the new dataframe
print("Shape of df_schema_train after removing NA values:", df_schema_train.shape)

File saved to /Users/ajaykallepalli/data/df_schema_train.csv
Shape of df_schema_train after removing NA values: (6016, 7)


In [16]:
# Calculate the number of rows per chunk
chunk_size = len(df_schema_train) // 10

# Base path for the CSV files
base_path = '/Users/ajaykallepalli/data/df_schema_train'

# Chunk the dataframe and save to CSV files
for i in range(10):
    start = i * chunk_size
    end = (i + 1) * chunk_size if i < 9 else len(df_schema_train)
    
    chunk = df_schema_train.iloc[start:end]
    
    # Create the file name with the suffix _1
    file_name = f"{base_path}_{i+1}.csv"
    
    # Save the chunk to CSV
    chunk.to_csv(file_name, index=False)
    print(f"Chunk {i+1} saved to {file_name}")

print("All chunks have been saved successfully.")


Chunk 1 saved to /Users/ajaykallepalli/data/df_schema_train_1.csv
Chunk 2 saved to /Users/ajaykallepalli/data/df_schema_train_2.csv
Chunk 3 saved to /Users/ajaykallepalli/data/df_schema_train_3.csv
Chunk 4 saved to /Users/ajaykallepalli/data/df_schema_train_4.csv
Chunk 5 saved to /Users/ajaykallepalli/data/df_schema_train_5.csv
Chunk 6 saved to /Users/ajaykallepalli/data/df_schema_train_6.csv
Chunk 7 saved to /Users/ajaykallepalli/data/df_schema_train_7.csv
Chunk 8 saved to /Users/ajaykallepalli/data/df_schema_train_8.csv
Chunk 9 saved to /Users/ajaykallepalli/data/df_schema_train_9.csv
Chunk 10 saved to /Users/ajaykallepalli/data/df_schema_train_10.csv
All chunks have been saved successfully.


In [ ]:
 
print("All chunks have been saved successfully.")

In [ ]:
max_instruction = 0
max_schema = 0
max_input = 0
max_output = 0
max_total = 0

instructions = df_schema_train["instruction"]
schemas = df_schema_train["db_schema"]
inputs = df_schema_train["question"]
outputs = df_schema_train["query"]

for instruction, schema, input, output in zip(instructions, schemas, inputs, outputs):
    # Count the number of tokens in the input data fields
    instruction_tokens = tokenizer.encode_plus(instruction, add_special_tokens=False, max_length=None)["input_ids"]
    schema_tokens = tokenizer.encode_plus(schema, add_special_tokens=False, max_length=None)["input_ids"]
    input_tokens = tokenizer.encode_plus(input, add_special_tokens=False, max_length=None)["input_ids"]
    output_tokens = tokenizer.encode_plus(output, add_special_tokens=False, max_length=None)["input_ids"]
    total_tokens = tokenizer.encode_plus(f"{instruction} {schema} {input} {output}", add_special_tokens=False, max_length=None)["input_ids"]

    max_instruction = max(max_instruction, len(instruction_tokens))
    max_schema = max(max_schema, len(schema_tokens))
    max_input = max(max_input, len(input_tokens))
    max_output = max(max_output, len(output_tokens))
    max_total = max(max_total, len(total_tokens))

# Display the results
print(f"\nMaximum token counts:")
print(f"Instruction: {max_instruction}")
print(f"Schema: {max_schema}")
print(f"Input: {max_input}")
print(f"Output: {max_output}")
print(f"Total: {max_total}")

# Add a small buffer to the maximum token count
buffer = 10
# max_seq_length can be set up to 2x the default context length
# of the base model because Unsloth supports RoPE Scaling internally.
# Here, we auto-configure this length based on input data analysis.
max_seq_length = max_total + buffer

# Display the table header
table_title = "Training Data Token Counts"
print(f"\n{table_title:-^70}")
print(f"{'Measure':<14}{'Instruction':<14}{'Schema':<14}{'Input':<14}{'Output':<14}{'Total':<14}")

# Display token counts in tabular form
print(f"{'Maximums':<14}{max_instruction:<14}{max_schema:<14}{max_input:<14}{max_output:<14}{max_total:<14}")
print(f"{'Max Seq Len':<14}{'':<14}{'':<14}{'':<14}{'':<14}{max_seq_length:<14}\n")

print(f"Set max_seq_length in FastLanguageModel to {max_seq_length} to handle the maximum number of tokens required by the input training data (Total Maximum + Buffer).")

# Add a small buffer to the maximum token count
buffer = 10
# max_seq_length can be set up to 2x the default context length
# of the base model because Unsloth supports RoPE Scaling internally.
# Here, we auto-configure this length based on input data analysis.
max_seq_length = max_qna + buffer

# Display the table header
table_title = "Training Data Token Counts"
print(f"\n{table_title:-^70}")
print(f"{'Measure':<14}{'Question':<14}{'Answer':<14}{'Combined':<14}")

# Display token counts in tabular form
print(f"{'Maximums':<14}{max_q:<14}{max_a:<14}{max_qna:<14}")
print(f"{'Max Seq Len':<14}{'':<14}{'':<14}{max_seq_length:<14}\n")

print(f"Set max_seq_length in FastLanguageModel to {max_seq_length} to handle the maximum number of tokens required by the input training data (Combined Maximum + Buffer).")